In [1]:
import tensorflow as tf
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import VGG16

In [3]:
# Path to your concrete damage dataset
dataset_path = 'sorted_images'

In [4]:
# Assuming your dataset is already divided into subfolders, each representing a type of concrete damage
damage_types = os.listdir(dataset_path)
print("Concrete Damage Types:", damage_types)

Concrete Damage Types: ['CorrosionStain', 'Crack', 'Efflorescence', 'ExposedBars', 'Spallation']


In [5]:
# ImageDataGenerator for data augmentation
datagen = ImageDataGenerator(rescale=1/255, validation_split=0.2,
                             rotation_range=10, width_shift_range=0.1, height_shift_range=0.1,
                             shear_range=0.1, zoom_range=0.1, horizontal_flip=True)

train_generator = datagen.flow_from_directory(dataset_path, target_size=(224,224), batch_size=4, 
                                              class_mode="categorical", subset='training')

validation_generator = datagen.flow_from_directory(dataset_path, target_size=(224,224), batch_size=4, 
                                              class_mode="categorical", subset='validation')


Found 9898 images belonging to 5 classes.
Found 2473 images belonging to 5 classes.


In [6]:
# Building a custom model
model = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.2),

    Conv2D(128, (3, 3), activation='relu'),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.2),

    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),

    Dense(len(damage_types), activation='softmax')
])

opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 220, 220, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 110, 110, 64)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 110, 110, 64)      0         
                                                                 
 conv2d_2 (Conv2D)           (None, 108, 108, 128)     73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 106, 106, 128)     147584    
                                                        

In [14]:
# Training the custom model
history = model.fit(train_generator, epochs=5, validation_data=validation_generator)


NameError: name 'train_generator' is not defined

In [7]:
from tensorflow.keras.layers import BatchNormalization, LeakyReLU
from tensorflow.keras.regularizers import l2

In [8]:
# Using VGG16 as base model
model_vgg16 = VGG16(input_shape=(224,224,3), include_top=False, weights='imagenet')
for layer in model_vgg16.layers:
    layer.trainable = True

# Adding custom layers on top of VGG16
# model = Sequential([
#     model_vgg16,
#     Flatten(),
#     Dense(512, activation='relu'),
#     Dropout(0.5),
#     Dense(len(damage_types), activation='softmax')
# ])


model = Sequential([
    model_vgg16,
    Flatten(),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(256, kernel_regularizer=l2(0.001)),  # Adding L2 regularization
    LeakyReLU(),  # Using LeakyReLU instead of 'relu'
    Dropout(0.3),
    Dense(len(damage_types), activation='softmax')
])

In [9]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten_1 (Flatten)         (None, 25088)             0         
                                                                 
 dense_2 (Dense)             (None, 512)               12845568  
                                                                 
 batch_normalization (BatchN  (None, 512)              2048      
 ormalization)                                                   
                                                                 
 dropout_3 (Dropout)         (None, 512)               0         
                                                                 
 dense_3 (Dense)             (None, 256)               131328    
                                                      

In [10]:
# Training the VGG16-based model
history = model.fit(train_generator, epochs=2, validation_data=validation_generator)


Epoch 1/2
2475/2475 [==============================] - 2909s 1s/step - loss: 1.7986 - accuracy: 0.2873 - val_loss: 2850842.5000 - val_accuracy: 0.2002
Epoch 2/2
2475/2475 [==============================] - 2843s 1s/step - loss: 1.5300 - accuracy: 0.3257 - val_loss: 9542935.0000 - val_accuracy: 0.2180


In [11]:
# Save and load the model
model.save('Concrete_Damage.h5')
concrete_damage_model = tf.keras.models.load_model('Concrete_Damage.h5')
concrete_damage_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten_1 (Flatten)         (None, 25088)             0         
                                                                 
 dense_2 (Dense)             (None, 512)               12845568  
                                                                 
 batch_normalization (BatchN  (None, 512)              2048      
 ormalization)                                                   
                                                                 
 dropout_3 (Dropout)         (None, 512)               0         
                                                                 
 dense_3 (Dense)             (None, 256)               131328    
                                                      

In [12]:
# Predicting with the model
import cv2

In [16]:
img_path = '\sorted_images\ExposedBars\image_0000192_crop_0000001.png'  # replace with path to your test image
img = cv2.imread(img_path)
# img = cv2.resize(img, (224, 224))
# img = np.reshape(img, [1, 224, 224, 3])
# img = img / 255.0

In [17]:
preds = concrete_damage_model.predict(img)
max_idx = np.argmax(preds)
class_indices = {v: k for k, v in train_generator.class_indices.items()}
predicted_class = class_indices[max_idx]
print("Predicted Concrete Damage Type:", predicted_class)


ValueError: Failed to find data adapter that can handle input: <class 'NoneType'>, <class 'NoneType'>